# O intuito desse notebook e recuperar todas as imagens disponiveis sobre pneumonia e covid  disponibilizadas nos diretorios do Git hub
e montas as pastas de imagens somente de covid e normal no dataset


## bibliotecas necessárias

In [27]:
!pip install urllib2

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


In [2]:
import pandas as pd
import random
import shutil
import os

## variaveis iniciais

In [3]:
def getDiretorio(tipo):
    retorno = ""
    if(tipo =="MaquinaLocal"):
        retorno = r'D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto'
    
    if(tipo =="GitHub"):
        retorno = r'https://github.com/ieee8023/covid-chestxray-dataset/blob/master/'

    if(tipo =="GoogleDrive"):
       retorno=r'/content/drive/My Drive/TCC Pos Graduação big data/TCC/Projeto/'
       
    
    return retorno

In [51]:
diretorio_raiz = getDiretorio("MaquinaLocal")
diretorio_dataset =r"{0}/{1}".format(diretorio_raiz,'dataset')
diretorio_imagem_covid =r"{0}/{1}".format(diretorio_dataset,'covid')
diretorio_pneumonia_normal =r"{0}/{1}".format(diretorio_dataset,'normal')

print(diretorio_raiz)
print(diretorio_dataset)
print(diretorio_imagem_covid)
print(diretorio_pneumonia_normal)




D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid
D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/normal


## cria os diretorios se não existirem

In [9]:
if not os.path.exists(diretorio_imagem_covid):
    os.mkdir(diretorio_imagem_covid)

if not os.path.exists(diretorio_pneumonia_normal):
    os.mkdir(diretorio_pneumonia_normal)

## variaveis do diretorio dos conjuntos de dados originais



## valida se o diretorio do git recebeu novas imagens


In [16]:
source_dir_metadata_local = r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/metadata.csv'

metadata_web_git ='https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv'



In [23]:
df_git_covid_web  = pd.read_csv(metadata_web_git,error_bad_lines=False)

df_git_covid_local = pd.read_csv(source_dir_metadata_local,error_bad_lines=False)





In [28]:

print(len(df_git_covid_web))
print(len(df_git_covid_local))

if(len(df_git_covid_web) > len(df_git_covid_local)):
    print('temos {0} imagens com covid a mais no git web'.format(len(df_git_covid_web) - len(df_git_covid_local)))
elif(len(df_git_covid_web) < len(df_git_covid_local)):
    print('foram removidas {0} imagens  no git web'.format(len(df_git_covid_local) - len(df_git_covid_web)))

  
  


565
565


In [ ]:
source_dir_kaggle_Covid =r'D:/Repos/datasetsGitHub/covid-chestxray-dataset/images'
source_dir_kaggle_pneumonia =r'D:/Repos/pneumonia-chest_xray'

In [12]:
df  = pd.read_csv(source_dir_metadata)

In [38]:
df_covid =  df[["location", "finding", "modality","filename","view"]]

df_covid.columns = ["localidade","prognostico","tipoExame","imagem","tipo"]


In [39]:
df_covid.head(2)

,localidade,prognostico,tipoExame,imagem,tipo
0,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",COVID-19,X-ray,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,PA
1,"Cho Ray Hospital, Ho Chi Minh City, Vietnam",COVID-19,X-ray,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,PA


## populando 

In [ ]:
labels =[]
lista_imagens_erro = []
lista_imagens_com_Covid = []
lista_imagens_com_pneumonia_normal = []
lista_tuplas_imagens =[]

## Importando nossas imagens para o banco de dados SqlLite

In [81]:
import sqlite3

In [96]:

sqliteConnection = sqlite3.connect("covid19.db")

In [98]:
def desconect_db(sqliteConnection):
    sqliteConnection.close()

In [99]:
def create_table(tableName):
    if(str(tableName).lower() =="ImagensCovid".lower()):
        sqliteConnection.execute('DROP TABLE IF EXISTS ImagensCovid;')
        sqliteConnection.execute('CREATE TABLE IF NOT EXISTS ImagensCovid(Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, Localidade TEXT,    '\
     'Prognostico TEXT, TipoExame TEXT, ImagemName  TEXT  UNIQUE,Tipo TEXT ,BlobData  BLOB)')



In [100]:
def convertToBinaryData(filename):
    with open(filename, 'rb') as file:
        blobData = file.read()
    return blobData

In [110]:
def insert_into_imagens(localidade, prognostico, tipoExame, imagemName,tipo,imagen_data):
    try:
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")
        sqlite_insert_blob_query = """ INSERT INTO ImagensCovid
        (Localidade, Prognostico, TipoExame, ImagemName,Tipo,BlobData) VALUES (?, ?, ?, ?,?,?)"""
        
        # Convert data into tuple format
        data_tuple = (localidade, prognostico, tipoExame, imagemName,tipo,imagen_data)
        cursor.execute(sqlite_insert_blob_query, data_tuple)
        sqliteConnection.commit()
        print("Image and file inserted successfully as a BLOB into a table")
        cursor.close()

    except sqlite3.Error as error:
        print(error)
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("the sqlite connection is closed")

In [121]:
sqliteConnection = sqlite3.connect("covid19.db")

In [123]:


# cursor = sqliteConnection.cursor()
# print("Connected to SQLite")
# sqlite_insert_blob_query = """ INSERT INTO Imagens
# (Localidade, Prognostico, TipoExame, ImagemName,Tipo,BlobData) VALUES (?, ?, ?, ?,?,?)"""


sqlite_insert_blob_query = """ INSERT INTO ImagensCovid(Localidade, Prognostico, TipoExame, ImagemName,Tipo,BlobData) VALUES (?, ?,?, ?,?,?)"""
       


localidade ='Cho Ray Hospital, Ho Chi Minh City, Vietnam'
prognostico ='COVID-19'
tipoExame ='X-ray',
imagemName ='auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg',
tipo ='PA',
blob_imagem = convertToBinaryData('D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid/auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg')

# Convert data into tuple format
data_tuple = (localidade, prognostico, tipoExame, imagemName,tipo,blob_imagem)

cursor = sqliteConnection.cursor()

cursor.execute(sqlite_insert_blob_query, data_tuple)

# sqliteConnection.commit()
# print("Image and file inserted successfully as a BLOB into a table")
# cursor.close()

InterfaceError: Error binding parameter 2 - probably unsupported type.

In [104]:
create_table('ImagensCovid')

In [113]:
        
lista_imagens_com_Covid =[]
quantidade = 0
for (i,row) in df_covid.head(1).iterrows():

        imagemName=row["imagem"]
        localidade =row["localidade"]
        prognostico =row["prognostico"]
        tipoExame =row["tipoExame"]
        tipo =row["tipo"]
        image_path  = "{0}/{1}".format(diretorio_imagem_covid,imagemName)
        # blob_imagem = convertToBinaryData(image_path)

        # insert_into_imagens(localidade,prognostico,tipoExame,imagemName,tipo,blob_imagem)
        print(image_path)
        # print(localidade)
        # print(prognostico)
        # print(tipoExame)
        # print(tipo)
        
        
        # image_copy_target= "{0}/{1}".format(diretorio_imagem_covid,file_name)
        # shutil.copy2(image_path,image_copy_target)
        # lista_imagens_com_Covid.append(image_copy_target)
        # quantidade +=1


        


D:/Repos/AprendendoPython/Pos_Graduacao_BigData/TCC/Projeto/dataset/covid/auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg


## listas auxiliares
